In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Apriori 
Apriori作为数据挖掘十大算法之一，专门挖掘特征之间的关联规则，例如购物篮尿布与啤酒之间的关系，关联规则有两个形式-frequent item以及关联法则，frequent item计算同一条数据项目间同时出现的频率，关联法则则是项目之间的关系。我们下面举个例子  

| Transaction number  | Items  |
|---|---|
| 0  | soy milk  |
| 1  | lettuce, diapers, wine, chard  |
| 2  | soy milk, diapers, wine, orange juice  |
| 3  | lettuce, soy milk, diapers, wine  |
| 4  | lettuce, soy milk, diapers, orange juice  |

在介绍frequent item之前，先理解两个词-support和confidence，我们定义含有该item的项目的比例称作item的support，例如soy milk的support是4/5，soy milk以及diapers的support是3/5。confidence是关联法则的词，例如{diapers}->{wine}，这个规则的confidence就会是support({diapers,wine})/({diapers})，也就是3/4=0.75。然而在做推荐之前，必须先建立frequent item以及关联法则，若item一多将会非常耗时，于是乎有了Apriori principle，简单来说就是如果一个itemset出现的频次不高，那么这个itemset的父集的频次也不高。

### 优缺点

优点：容易复现、方便并行计算   
缺点：数据量大则会很慢


接下来我们进行示例

In [2]:
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return list(map(frozenset, C1))#use frozen set so we
                            #can use it as a key in a dict    

def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt.keys(): ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

dataSet=loadDataSet()
print(dataSet)
C1=createC1(dataSet)
print(C1)
D=[n for n in map(set,dataSet)]
print(D)
L1,suppData0=scanD(D,C1,0.5)
L1

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]
[frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4}), frozenset({5})]
[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]


[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5})]

In [3]:
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
    return retList

def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

L,suppData=apriori(dataSet)
print(L)
aprioriGen(L[0],2)

[[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5})], [frozenset({3, 5}), frozenset({1, 3}), frozenset({2, 5}), frozenset({2, 3})], [frozenset({2, 3, 5})], []]


[frozenset({1, 3}),
 frozenset({1, 2}),
 frozenset({1, 5}),
 frozenset({2, 3}),
 frozenset({3, 5}),
 frozenset({2, 5})]

In [4]:
L,suppData=apriori(dataSet,minSupport=0.7)
L

[[frozenset({3}), frozenset({2}), frozenset({5})], [frozenset({2, 5})], []]

计算出support，就可以看数据之间有什么关联法则，例如有个高频的itemset{soy milk, lettuce}，那么或许可以建立一个关联法则{soy milk}->{lettuce}，但不一定代表{lettuce}->{soy milk}，接下来来看如何发现item之间的关联法则，我们会用到confidence的概念  

对于一个法则P->H，其confidence为support(P|H)/support(P)，‘|’不是条件，而是联集的意思，根据之前建立的support，我们可以轻易的计算出所有关联法则，我们仍然会用到apriori的概念来降低计算量，若有个关联法则R无法满足最低confidence的阈值，那么他的子集的confidence也不会满足最低confidence阈值，例如有个关联法则{0,1,2}->{3}不满足，那么subset{0,1,2}->{3}也不会满足

In [5]:
def generateRules(L, supportData, minConf=0.7):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(1, len(L)):#only get the sets with two or more items
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList         

def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print(freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

# merge function
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

L,suppData=apriori(dataSet,minSupport=0.5)
rules=generateRules(L,suppData,minConf=0.7)
rules

frozenset({1}) --> frozenset({3}) conf: 1.0
frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0


[(frozenset({1}), frozenset({3}), 1.0),
 (frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0)]

In [6]:
rules=generateRules(L,suppData,minConf=0.5)
rules

frozenset({5}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({5}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({1}) conf: 0.6666666666666666
frozenset({1}) --> frozenset({3}) conf: 1.0
frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({3}) --> frozenset({2}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({5}) --> frozenset({2, 3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({2, 5}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3, 5}) conf: 0.6666666666666666


[(frozenset({5}), frozenset({3}), 0.6666666666666666),
 (frozenset({3}), frozenset({5}), 0.6666666666666666),
 (frozenset({3}), frozenset({1}), 0.6666666666666666),
 (frozenset({1}), frozenset({3}), 1.0),
 (frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({3}), frozenset({2}), 0.6666666666666666),
 (frozenset({2}), frozenset({3}), 0.6666666666666666),
 (frozenset({5}), frozenset({2, 3}), 0.6666666666666666),
 (frozenset({3}), frozenset({2, 5}), 0.6666666666666666),
 (frozenset({2}), frozenset({3, 5}), 0.6666666666666666)]